In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image

In [ ]:
%matplotlib inline

# Gradient Boosting Machines and the HistGradientBoostingClassifier
Over the past few years, a class of models known as Gradient Boosting Machines (GBM) have become quite popular. One particular version/implementation known as [XGBoost](https://github.com/dmlc/xgboost) is commonly the technique of choice in winning Kaggle competitions. While these methods can seem quite mysterious and it's true that under the hood there are a lot of mathematical and technical details, the general idea behind them is pretty easy to grasp:

* Build some model to predict something and make predictions
* Compute the residuals from your first model (the errors)
* Build a second model to predict *the first model's errors*
* **Add** the second model's prediction to the first model's prediction (the boosting part)
* ... rinse and repeat

I emphased **Add** because boosting is an *additive* approach to modeling - we add a bunch of simple models together to make a more complex model.

So, the big idea is iteratively improving the overall model by adding new models that try to better predict the previous model's errors. A good first read is this blog post, [Gradient Boosting Explained](https://www.gormanalysis.com/blog/gradient-boosting-explained/) by Ben Gorman (a Kaggle wizard). These techniques can be applied to both regression problems and classification problems. Then move on to the [brilliantly readable set of explanatory articles](https://explained.ai/gradient-boosting/) on GBM's done by Terrance Parr and Jeremy Howard.

Even if you don't end up getting into the math details, it's good to keep in mind that when we say "build a model" we are implicitly saying that we need to do some sort of optimization to find the best model parameter values to minimize some *loss function* - a measure of overall error in our model predictions (for regression this might be MSE and for classification, [cross-entropy](https://en.wikipedia.org/wiki/Cross_entropy)). In these boosting models, the general optimization technique known as *stochastic gradient descent* often plays an important role. Hey, we came across this idea earlier when discussing the various solvers (e.g. 'saga') available in `LogisticRegression` in sklearn. So, developing some general intutition about how SGD works can help you understand and develop intuition about various machine learning techniques ranging from logistic regression to GBM's. We don't have the time to get into the details in this short summer course, but I will provide additional pointers to some very readable, and high quality, explanations, tutorials and demonstrations of these concepts. I'll do that at the end of this notebook.

For now, let's just naively use a specific GBM that has recently been added to sklearn known as a [Histogram-based Gradient Boosting Classification Tree](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.HistGradientBoostingClassifier.html?highlight=histgradient#sklearn.ensemble.HistGradientBoostingClassifier). Interestingly, this classifier is based on one called [LightGBM which was developed by Microsoft](https://github.com/Microsoft/LightGBM) and released as an open source project. As you'll see from the `import` statements below, it's still considered an experimental feature in sklearn. In order to use it, there were a few details that needed ironing out:

* The target variable needs to be encoded using the [OrdinalEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html) though this seems to happen automatically (?).
* The sklearn preprocessing steps we have been doing actually lead to the columns being reordered by column type (categorical vs numeric) and this led to our `categorical_cols_idx` list being wrong when we fed it to the `HistGradientBoostingClassifier`. You'll see comments in the code below that address issue.



In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix

# explicitly require this experimental feature
from sklearn.experimental import enable_hist_gradient_boosting  # noqa
# now you can import normally from ensemble
from sklearn.ensemble import HistGradientBoostingClassifier

# Need the following to encode our target variable
from sklearn.preprocessing import OrdinalEncoder

Ok, let's use this thing.

In [ ]:
train_X = pd.read_csv('./data/train_x.csv')
train_y_raw = pd.read_csv("./data/raw/train_y.csv")
test_X = pd.read_csv("./data/test_x.csv")

# Drop id from train_y_raw
train_y = train_y_raw.iloc[:, 1]

# Create lists of columns by variable type
target_col = ['status_group']
categorical_cols = train_X.select_dtypes(include=['object']).columns.tolist()
numeric_cols = train_X.select_dtypes(include=['number']).columns.tolist()


In [ ]:
categorical_cols

In [ ]:
numeric_transformer_hgbc = StandardScaler()
categorical_transformer_hgbc = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=np.nan)
target_transformer_hgbc = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=np.nan)

preprocessor_hgbc = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer_hgbc, categorical_cols),
        ('num', numeric_transformer_hgbc, numeric_cols)],
        remainder='passthrough')

# The preprocessor reorders the columns into blocks by type as defined by the transformers. So, now
# the categorical cols are the first columns. We need their index numbers to use in the call to the
# HistGradientBoostingClassifier() so it knows which cols to treat as categoricals (even though they
# have been transformed into meaningless integers.)

categorical_cols_idx = [_ for _ in range(len(categorical_cols))]

# Append classifier to preprocessing pipeline.
clf_hgbc = Pipeline(steps=[('preprocessor', preprocessor_hgbc),
                      ('classifier', HistGradientBoostingClassifier(categorical_features=categorical_cols_idx))])

# Fit model
clf_hgbc.fit(train_X, train_y)
print(f"hgbc training score: {clf_hgbc.score(train_X, train_y):.3f}")

# Can just do the prediction using test data and predict will send it through the pipeline for preprocessing.
clf_hgbc_test = clf_hgbc.predict(test_X)

# Create submission file 
submit_dict = {'id': test_X['id'],
              'status_group': clf_hgbc_test}

hgbc_1_submission = pd.DataFrame(submit_dict, columns=['id', 'status_group'])
hgbc_1_submission.to_csv('./output/hgbc_1_submission.csv', index=False)


Well, hgbc did better than logistic model on the test data when I submitted it to Pump it Up (test accuracy = 0.7960). Still not as good as the random forest.

Let's look at confusion matrix to see where we are making mistakes.

In [ ]:
plot_confusion_matrix(clf_hgbc, train_X, train_y, labels=clf_hgbc['classifier'].classes_)

Not surprisingly, the 'functional needs repair' status is the most difficult to predict correctly for this model as well.

# Ensembles
Well, we've built three pretty different types of models: logistic regression, random forest, and a GBM. Why not bake them into a nice ensemble cake and see if we can improve our overall accuracy on the test data. Keep in mind that both random forests and GBM's are already ensemble approaches. So we are really making ensembles of ensembles!

I saved the logistic regression and random forest models in a pickle file. Let's get them.

In [ ]:
import pickle

In [ ]:
with open("./output/models_clf_rf.pkl", "rb") as model_file:
    models_clf_rf = pickle.load(model_file)

In [ ]:
models_clf_rf.keys()

In [ ]:
from sklearn.ensemble import VotingClassifier

In [ ]:
# 1. Create model
ensemble_1 = VotingClassifier(estimators=[('clf_lr', models_clf_rf['clf_LR']),
                                          ('clf_rf', models_clf_rf['clf_RF']),
                                          ('clf_hgbc', clf_hgbc)], 
                              voting='soft', weights=[1.0, 1.0, 1.0])

# 2. Fit model
ensemble_1.fit(train_X, train_y)
print(f"Score: {ensemble_1.score(train_X, train_y):.4f}")

# 3. Predict
clf_ensemble_1_test = ensemble_1.predict(test_X)

# Create submission file 
submit_dict = {'id': test_X['id'],
              'status_group': clf_ensemble_1_test}

clf_ensemble_1_submission = pd.DataFrame(submit_dict, columns=['id', 'status_group'])
clf_ensemble_1_submission.to_csv('./output/clf_ensemble_1_submission.csv', index=False)

Hmmm, pretty high training score. Going to be interesting to see how we do on the contest test data...

It scored 0.8104 in the actual competition - best submission so far. Lots of room for improvement and likely need to focus on variable selection, data prep and feature engineering. Unlikely that model tweaks are going to help.

## Learning More

### Gradient boosting machines

If you want to learn more about GBM's and gradient descent, here's a few things you can do. Start by visiting Parr and Howard's explanation page for GBM's:

* https://explained.ai/gradient-boosting/ - great portal to several readable overviews, background papers, and Jupyter notebooks

They go on in a series of really nice posts to explain how GBM's work:

* [Distance to Target](https://explained.ai/gradient-boosting/L2-loss.html) - Develops the intuition behind additive models (such as boosting) and works through a few examples to show how GBM's work.
* [Heading in the right direction](https://explained.ai/gradient-boosting/L1-loss.html) - Takes a closer look at how GBMs work such as using the sign instead of the magnitude of the residual to guide the optimization.
* [Gradient boosting performs gradient descent](https://explained.ai/gradient-boosting/descent.html) - They reveal that GBM's are really using *gradient descent*, a fundamental optimization technique and help you develop intution around how it works - if you're trying to get down to the bottom of valley quickly, in which direction do you start walking?

The authors have created a series of Jupyter notebooks that go along with their tutorials.